In [238]:
# %pip install git+https://github.com/tensorflow/docs

import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [239]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.13.453.3037' # '12.12.450.4196' '12.13.453.3037' Version 12.12.448.6653 12.11.446.9344 Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>
RANKED_ID = 1100    # 1090 normal game 1100 ranked game
PATCH: date = date(2022, 7, 16)  # date(2022, 7, 1) date(2022, 7, 16)

TARGETNAME = 'placement'

In [240]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [241]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,jUVgWLecKv7AQD3mNu-6B4sADn9QKlqt5eQ6Jf_0VHUZTFo,OCChXvLDGG4GlJ2JnwT74iIKgfO42kT1Weo2jiTz7EgESPQ,53sAlVAf0iHz6Jfu6zPSXuNdDIybeSVAR0pI3671QyEheQ64z7pfvXAhEQ4YmXJ7fshpabdMwkmnwQ,Thwackey,4568,1658010769000,329,jUVgWLecKv7AQD3mNu-6B4sADn9QKlqt5eQ6Jf_0VHUZTFo,Thwackey,335,I,167,132,False,False,True,False
1,rPqSTsimwSeqQaKaNSOwmgZhj0OkbsAvYDeeAoAnhXc11bk,6EKn7VafcxRqRnqQZpFqA8bz_KFgufoH0IJXIH057pd-Ww,0qftKvhpbwj4f7FJv5M5McEkeEladDnEu_VsdQbfy39tUI9pt7UA4LKQBT9nQDe40oeZK3rdchlhOg,LittleBlindMoose,677,1658011511027,144,rPqSTsimwSeqQaKaNSOwmgZhj0OkbsAvYDeeAoAnhXc11bk,LittleBlindMoose,287,I,127,94,False,False,True,False
2,nyFl_kl7fMBWYpAXVRzBb-mkdgzPjICqppKiMBG5w1kbofs,siaQBA1dB3RNM3PMlA7tiMz-w6MimTlfevN0FVE68pBmmdQ,wOggHY9LjPSDZ62B8s_OE3VqL2EuxvdZ8jLexYiqwioCkLlyDmo3UY8RbT5MGU_0Wk9Wo7VdwKu5ig,HARDSTYL3,5431,1658003365000,92,nyFl_kl7fMBWYpAXVRzBb-mkdgzPjICqppKiMBG5w1kbofs,HARDSTYL3,327,I,174,134,False,False,True,True
3,8QicLiX6twjbaDeu6_noCrkKs7T673Wgqv5_tuwRTXpWysk,12CpnHv-P-GMDrNcjqtZsyFyDjfnGxV4EUaT8_RFGd02ifc,9xXRgKH3S0EHoaP1Ee2nAtjICKvfQPBhqKoIqXr0Wpcy_lAuT5UziHIEmIjPldEuqi-hV8NE6uarHg,Neo LAN,4244,1657999200000,124,8QicLiX6twjbaDeu6_noCrkKs7T673Wgqv5_tuwRTXpWysk,Neo LAN,435,I,65,40,False,False,True,False
4,mI3egzjwdGTDKt5_lBMWWkx1WlzrUl_BMvsJvdUFOys_-9I,dU39wcNQnKBDa1pEDWKi727fqqw6JpxNTNX6AbGUh7AOFWs,NJUMPpu7BdxU81sSulUfXntsIktkZHu8As5APFwQEioqzsXcleS2VTfYXkbK40lOV6AFsIautWXW1Q,Bradini,4626,1657914584000,313,mI3egzjwdGTDKt5_lBMWWkx1WlzrUl_BMvsJvdUFOys_-9I,Bradini,438,I,256,225,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,wOl5iKlDAermNYtwN41ftRN0qFzXNNCAMZlt9VRqAy06o9c,lCBGXPoEAFml-rvkCmvo4EDbFOv8Wp_SBvYsfBG5yedN1A,20HcCpfGQ5v2Zhi-mX465x4WW0Lu55Xp1PRSiLO5-8Tc4jvni-Xp9XOvb4aSpJI57OagjEgepyy0tw,Rolling ZoZo,588,1658013745571,157,wOl5iKlDAermNYtwN41ftRN0qFzXNNCAMZlt9VRqAy06o9c,Rolling ZoZo,243,I,125,100,False,False,True,False
496,Dgv2J9sAErASwMrtbp6xKHUT8rZhOkjm__znyavH2GZp9tpcjBFomf9KHw,LLGEY_8lF-2iPfKJ3YzdbDld7WOH6k-msh7aHeWhcHc1oa7cbVDZvoOH,cdbqXVnh1j0pflJ4LkiCe02UdAsBwiCPl-VR-Vrnmz_5wMDkAN0RMAlJGZQEQjfHGVzJ9omyr_tZxQ,Rarakko,23,1657986154000,15,Dgv2J9sAErASwMrtbp6xKHUT8rZhOkjm__znyavH2GZp9tpcjBFomf9KHw,Rarakko,361,I,220,192,False,False,True,False
497,N_VhJNeAZnSPiAT2t5gqchFdgR15JFX1zjZBBH3-cxu5wsg,giufqtfzVYuKSgnu5cxkfeYLqXnWdQMDYcuCFUAcEWrJ0cM,i22M_GAs966g7KDqzqM_62UEJUSp76zwXk55o-xmqj7IXnZ7q8wihySwEGQirZ-iLOF_lPm4MRzdnw,Ghostnomnom,4379,1658007616423,521,N_VhJNeAZnSPiAT2t5gqchFdgR15JFX1zjZBBH3-cxu5wsg,Ghostnomnom,363,I,110,70,False,False,False,False
498,naXRULIrhL8fsi5eD-5jel90glJeQ8xTpE8KzMzVaydgnyHP,D8ngZMbpD1-TsXzRYz7OpeubgjjsFKSZi77sYIJZHiYpXF3nqo49FYJt,DTP400UCwNQ2v2eo9wtBR8FGd64eWHwWxx99eA-KraMI7YEnTqnzcLSVRH0wV8tfJNPmib1O5IFDjg,vvoody,5411,1658004204000,30,naXRULIrhL8fsi5eD-5jel90glJeQ8xTpE8KzMzVaydgnyHP,vvoody,178,I,109,84,False,False,False,False


# Load unique matches id

In [242]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

2022-07-17 11:26:48,623 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Sleepystep.json.gz'


In [243]:
len(uniq_matches_id)

17686

In [244]:
len(matches_asset)

20333

In [245]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [246]:
len(uniq_matches)

17686

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [247]:
matches_asset[-1]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [248]:
matches_asset[-1]['info']['queue_id']

1100

In [249]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [250]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [251]:
timedelta(days=3)

datetime.timedelta(days=3)

In [252]:
latest_3d_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                     and ((datetime.now() - timedelta(days=3)) <= datetime.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [253]:
len(latest_matches)

3203

In [254]:
len(latest_patch_matches)

1445

In [255]:
len(latest_3d_matches)

2995

# Process api details to datasets rows

In [256]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [257]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)
matches_3d_array = process_matches(latest_3d_matches)

In [258]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4375685979',
 'placement': 7,
 'augment0': 'TFT7_Augment_GadgetExpert',
 'augment1': 'TFT6_Augment_TradeSectorPlus',
 'augment2': 'TFT6_Augment_MaxLevel10',
 'Set7_Cannoneer': 2,
 'Set7_Dragon': 1,
 'Set7_Evoker': 1,
 'Set7_Guardian': 1,
 'Set7_Mystic': 0,
 'Set7_Revel': 2,
 'Set7_Shimmerscale': 1,
 'Set7_Trainer': 1,
 'Set7_Whispers': 0,
 'TFT7_Thresh': 1,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Jinx': 2,
 'TFT7_Jinx_item0': 'StatikkShiv',
 'TFT7_Jinx_item1': 'Deathblade',
 'TFT7_Tristana': 2,
 'TFT7_Lulu': 2,
 'TFT7_Lulu_item0': 'ArchangelsStaff',
 'TFT7_Corki': 1,
 'TFT7_Corki_item0': 'Quicksilver',
 'TFT7_Corki_item1': 'InfinityEdge',
 'TFT7_Corki_item2': 'LastWhisper',
 'TFT7_Sona': 2,
 'TFT7_DragonGold': 2,
 'TFT7_DragonGold_item0': 'ShimmerscaleMogulsMail',
 'TFT7_DragonGold_item1': 'RedBuff',
 'TFT7_DragonGold_item2': 'WarmogsArmor',
 'TFT7_TrainerDragon': 2,
 'TFT7_TrainerDragon_item0': 'TrainerSnax'}

In [259]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)
matches_league_3d_df = pd.json_normalize(matches_3d_array)

In [260]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,Set7_Mystic,...,TFT7_Karma_item2,TFT7_Taric_item1,TFT7_Braum_item2,TFT7_Taric_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Twitch_item2,TFT7_Jinx_item2,TFT7_Sejuani_item2,augment3
0,NA1_4375685979,7,TFT7_Augment_GadgetExpert,TFT6_Augment_TradeSectorPlus,TFT6_Augment_MaxLevel10,2.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4375685979,4,TFT6_Augment_Featherweights2,TFT6_Augment_RichGetRicherPlus,TFT6_Augment_RadiantRelics,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4375685979,1,TFT6_Augment_SecondWind2,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_ThinkFast,NaN,1.0,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4375685979,8,TFT6_Augment_CyberneticImplants2,TFT6_Augment_CyberneticUplink2,TFT7_Augment_CursedCrown,0.0,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4375685979,5,TFT6_Augment_MetabolicAccelerator,TFT6_Augment_BlueBattery2,TFT6_Augment_RadiantRelics,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25619,NA1_4373238726,7,TFT6_Augment_SunfireBoard,TFT6_Augment_CelestialBlessing3,TFT6_Augment_FuturePeepers,NaN,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25620,NA1_4373238726,8,TFT6_Augment_CyberneticShell2,TFT7_Augment_CursedCrown,TFT7_Augment_SwiftshotPressTheAttack,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25621,NA1_4373238726,4,TFT7_Augment_MageConference,TFT6_Augment_LudensEcho3,TFT6_Augment_CelestialBlessing2,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25622,NA1_4373238726,2,TFT6_Augment_CyberneticImplants2,TFT6_Augment_CelestialBlessing3,TFT6_Augment_CelestialBlessing2,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [261]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Cannoneer,Set7_Dragon,Set7_Evoker,Set7_Guardian,Set7_Mystic,...,TFT7_Karma_item2,TFT7_Taric_item1,TFT7_Braum_item2,TFT7_Taric_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Twitch_item2,TFT7_Jinx_item2,TFT7_Sejuani_item2,augment3
0,NA1_4375685979,7,TFT7_Augment_GadgetExpert,TFT6_Augment_TradeSectorPlus,TFT6_Augment_MaxLevel10,2.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4375685979,4,TFT6_Augment_Featherweights2,TFT6_Augment_RichGetRicherPlus,TFT6_Augment_RadiantRelics,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4375685979,1,TFT6_Augment_SecondWind2,TFT6_Augment_ThrillOfTheHunt2,TFT7_Augment_ThinkFast,NaN,1.0,1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4375685979,8,TFT6_Augment_CyberneticImplants2,TFT6_Augment_CyberneticUplink2,TFT7_Augment_CursedCrown,0.0,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4375685979,5,TFT6_Augment_MetabolicAccelerator,TFT6_Augment_BlueBattery2,TFT6_Augment_RadiantRelics,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25619,NA1_4373238726,7,TFT6_Augment_SunfireBoard,TFT6_Augment_CelestialBlessing3,TFT6_Augment_FuturePeepers,NaN,NaN,0.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25620,NA1_4373238726,8,TFT6_Augment_CyberneticShell2,TFT7_Augment_CursedCrown,TFT7_Augment_SwiftshotPressTheAttack,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25621,NA1_4373238726,4,TFT7_Augment_MageConference,TFT6_Augment_LudensEcho3,TFT6_Augment_CelestialBlessing2,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25622,NA1_4373238726,2,TFT6_Augment_CyberneticImplants2,TFT6_Augment_CelestialBlessing3,TFT6_Augment_CelestialBlessing2,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [262]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [263]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)
matches_league_3d_df = reorder_df_col(matches_league_3d_df)

# Output dataframes

In [264]:
# matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
# matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)
matches_league_3d_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.pickle'))
matches_league_3d_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.csv'), index=False)

# End